In [ ]:
#enhanced_recommender.py
# 30.04

import os
import json
import math
import time
import random
from collections import defaultdict

import numpy as np
import pandas as pd
import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark   = False

import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from transformers import BertModel, BertConfig, get_cosine_schedule_with_warmup
from captum.attr import IntegratedGradients
import shap
import scipy.stats as stats
import matplotlib.pyplot as plt

# ---- helper functions for attention-entropy & plots ----
def compute_layer_head_entropies(attn_weights, layer):
    W = attn_weights[layer]            # (B, H, L, L)
    avg = W.mean(dim=0).mean(dim=1)    # (H, L)
    H, L = avg.shape
    ents = []
    for h in range(H):
        p = avg[h] / (avg[h].sum() + 1e-12)
        ent = -(p * torch.log(p + 1e-12)).sum().item() / math.log(L)
        ents.append(ent)
    hmax_idx = int(np.argmax(ents))
    hmin_idx = int(np.argmin(ents))
    return ents, hmax_idx, hmin_idx

def plot_layer_entropy(attn_weights, layer):
    ents, _, _ = compute_layer_head_entropies(attn_weights, layer)
    plt.figure()
    plt.plot(range(1, len(ents)+1), ents, marker='o')
    plt.xlabel("Head")
    plt.ylabel("Normalized Entropy")
    plt.title(f"Layer {layer+1} Head Entropies")
    plt.tight_layout()
    plt.show()

def plot_attention_heatmap(attn_weights, layer, head_idx, title=""):
    heatmap = attn_weights[layer][0, head_idx].detach().cpu().numpy()
    plt.figure()
    plt.imshow(heatmap, aspect='auto')
    plt.colorbar()
    plt.title(f"Layer {layer+1} Head {head_idx+1} Attention {title}")
    plt.xlabel("Key Position")
    plt.ylabel("Query Position")
    plt.tight_layout()
    plt.show()

# === Reproducibility Helper ===
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark   = False

# === Bootstrap CI helper ===
def bootstrap_ci(data, n_bootstrap=1000, alpha=0.05):
    boot_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        boot_means.append(np.mean(sample))
    lower = np.percentile(boot_means, 100 * alpha/2)
    upper = np.percentile(boot_means, 100 * (1 - alpha/2))
    return lower, upper, np.mean(data)

# === Utility functions ===
def safe_float(x, default=0.0):
    try:
        return float(x)
    except:
        return default

def load_json_file(path):
    if not os.path.exists(path):
        return {}
    content = open(path, 'r', encoding='utf-8').read().strip()
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        out = {}
        for line in content.splitlines():
            try:
                obj = json.loads(line)
                if isinstance(obj, dict):
                    out.update(obj)
            except:
                continue
        return out

def merge_activity_features(paths):
    combined = {}
    for p in paths:
        data = load_json_file(p)
        for k, feat in data.items():
            vals = feat if isinstance(feat, (list, tuple)) else [feat]
            vals = [safe_float(x) for x in vals]
            if k in combined:
                combined[k] = [(a + b) / 2 for a, b in zip(combined[k], vals)]
            else:
                combined[k] = vals
    return combined

def build_feature_matrix(json_data, id_map, feat_dim=32):
    mat = torch.zeros(len(id_map), feat_dim)
    for key, idx in id_map.items():
        vec = torch.tensor([safe_float(x) for x in json_data.get(key, [])], dtype=torch.float32)
        vec = torch.cat([vec, torch.zeros(max(0, feat_dim - vec.numel()))])[:feat_dim]
        mat[idx] = vec
    return mat

def get_batch_side_info(seqs, feat_matrix):
    B, L = seqs.shape
    out = []
    for i in range(B):
        mask = seqs[i] != 0
        if mask.sum() > 0:
            out.append(feat_matrix[seqs[i][mask]].mean(0))
        else:
            out.append(torch.zeros(feat_matrix.size(1), device=feat_matrix.device))
    return torch.stack(out, 0)

# === Dataset & Time2Vec ===
class Time2Vec(nn.Module):
    def __init__(self, in_f, out_f):
        super().__init__()
        self.lin = nn.Linear(in_f, 1)
        self.per = nn.Linear(in_f, out_f - 1)
    def forward(self, t):
        return torch.cat([self.lin(t), torch.sin(self.per(t))], -1)

class MOOCubeCourseDataset(Dataset):
    def __init__(self, csv_path, min_seq_len=2, max_seq_len=50, min_interactions=5, split="train"):
        df = pd.read_csv(csv_path, header=None, names=["user","item","time"]).sort_values(["user","time"])
        df["dt"] = df.groupby("user")["time"].diff().fillna(0)
        df["dt"] /= df["dt"].max() or 1
        self.u2i, self.i2i = {}, {}
        df["u_idx"] = df.user.map(lambda x: self.u2i.setdefault(x, len(self.u2i)))
        df["i_idx"] = df.item.map(lambda x: self.i2i.setdefault(x, len(self.i2i)))
        grp = df.groupby("u_idx")[['i_idx','dt']].apply(lambda g: list(zip(g.i_idx, g.dt)))
        grp = grp[grp.map(len) >= min_interactions]
        self.data = []
        for seq in grp:
            L = len(seq); tcut, vcut = int(0.8*L), int(0.9*L)
            if split=="train" and 2 <= tcut < L:
                inp, tgt = seq[:tcut], seq[tcut][0]
            elif split=="val" and 2 <= vcut < L:
                inp, tgt = seq[:vcut], seq[vcut][0]
            elif split=="test" and L-1 >= 2:
                inp, tgt = seq[:-1], seq[-1][0]
            else:
                continue
            if len(inp) > max_seq_len:
                inp = inp[-max_seq_len:]
            self.data.append((
                torch.tensor([i for i,_ in inp], dtype=torch.long),
                torch.tensor([d for _,d in inp], dtype=torch.float32),
                torch.tensor(tgt, dtype=torch.long)
            ))
        print(f"[{split}] {len(self.data)} sequences")

    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

class EnhancedMOOCDataset(MOOCubeCourseDataset):
    def __init__(self, *args, neg_samples=5, **kwargs):
        super().__init__(*args, **kwargs)
        self.neg_samples = neg_samples
        pop = defaultdict(int)
        for seq,_,t in self.data:
            for i in seq: pop[i.item()] += 1
        top = sorted(pop.items(), key=lambda x:-x[1])[:2000]
        self.pop_items   = [i for i,_ in top]
        counts = torch.tensor([c for _,c in top], dtype=torch.float)
        self.pop_weights = counts / counts.sum()

    def __getitem__(self, idx):
        seq, dts, tgt = super().__getitem__(idx)
        w = self.pop_weights.clone()
        if tgt.item() in self.pop_items:
            w[self.pop_items.index(tgt.item())] = 0
        w = w / w.sum() if w.sum()>0 else w
        negs = torch.multinomial(w, self.neg_samples, replacement=False)
        negs = torch.tensor(self.pop_items, dtype=torch.long)[negs]
        return seq, dts, tgt, negs

def collate_fn(batch):
    M = max(len(x[0]) for x in batch)
    out = []
    for it in batch:
        s,d,t = it[0], it[1], it[2]
        pad = M - len(s)
        s = torch.cat([s, torch.zeros(pad, dtype=torch.long)])
        d = torch.cat([d, torch.zeros(pad)])
        if len(it)==4:
            out.append((s,d,t,it[3]))
        else:
            out.append((s,d,t))
    return tuple(torch.stack(cols) for cols in zip(*out))

# === Side-Info Processors & Disentanglement ===
class EnhancedSideInfoProcessor(nn.Module):
    def __init__(self, dims, d_model, dropout=0.1):
        super().__init__()
        self.proj = nn.ModuleDict({
            n: nn.Sequential(
                nn.Linear(dim, d_model*2), nn.GELU(),
                nn.Linear(d_model*2, d_model), nn.LayerNorm(d_model)
            ) for n, dim in dims.items()
        })
        self.attn = nn.MultiheadAttention(d_model, 8, batch_first=True)
        self.ffn  = nn.Sequential(
            nn.Linear(d_model, d_model*4), nn.GELU(),
            nn.Linear(d_model*4, d_model), nn.Dropout(dropout)
        )
        self.ln   = nn.LayerNorm(d_model)

    def forward(self, feats):
        P = [self.proj[n](f) for n, f in feats.items()]
        F = torch.stack(P, 1)
        A, _ = self.attn(F, F, F)
        H = self.ln(A + F)
        return self.ffn(H) + H

class CrossModalDisentanglement(nn.Module):
    def __init__(self, dims, d_model):
        super().__init__()
        self.priv = nn.ModuleDict({
            n: nn.Sequential(nn.Linear(dim, d_model), nn.ReLU(), nn.LayerNorm(d_model))
            for n, dim in dims.items()
        })
        total = len(dims) * d_model
        self.shared = nn.Sequential(nn.Linear(total, d_model), nn.ReLU(), nn.LayerNorm(d_model))

    def forward(self, feats):
        privates = [self.priv[n](f) for n, f in feats.items()]
        cat      = torch.cat(privates, -1)
        sh       = self.shared(cat)
        return sh, privates

def orthogonality_loss(privates):
    if len(privates) < 2:
        return 0.0
    loss, n = 0.0, len(privates)
    for i in range(n):
        for j in range(i+1, n):
            loss += (privates[i] * privates[j]).sum(-1).mean()
    return loss / (n*(n-1)/2)

# === Model & Loss ===
class EnhancedFusionRecommenderWithDisentanglement(nn.Module):
    def __init__(self, num_items, d_model=768, side_info_dims={}, max_seq_len=50,
                 lambda_aux=0.5, mask_rate=0.15, ablation=None):
        super().__init__()
        self.ablation    = ablation
        self.lambda_aux  = lambda_aux
        self.mask_rate   = 0.0 if ablation=="no_masked" else mask_rate
        self.emb_item = nn.Embedding(num_items, d_model, padding_idx=0)
        self.emb_pos  = nn.Embedding(max_seq_len, d_model)
        self.ortho_weight = 0.0 if ablation=="no_orthogonality" else 0.1
        if ablation == "no_time":
            self.emb_time = lambda t: torch.zeros(t.size(0), t.size(1), d_model, device=t.device)
        else:
            self.emb_time = Time2Vec(1, d_model)
        self.side_proc = EnhancedSideInfoProcessor(side_info_dims, d_model)
        if ablation == "no_disentanglement":
            self.disent = None
        else:
            self.disent = CrossModalDisentanglement(side_info_dims, d_model)
        self.side_attn    = nn.MultiheadAttention(d_model, 2, batch_first=True)
        cfg = BertConfig(
            hidden_size=d_model, num_hidden_layers=8,
            num_attention_heads=8, intermediate_size=d_model*4,
            hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1
        )
        self.transformer  = BertModel(cfg)
        self.proj_head    = nn.Sequential(nn.Linear(d_model,d_model), nn.GELU(), nn.Linear(d_model,d_model))
        self.pred_head    = nn.Linear(d_model, num_items)
        self.mask_head    = nn.Linear(d_model, num_items)
        self.tau          = nn.Parameter(torch.tensor(0.07))

    def forward(self, seqs, dts, side_info=None):
        B, L = seqs.shape
        pos = torch.arange(L, device=seqs.device).unsqueeze(0)
        x   = self.emb_item(seqs) + self.emb_pos(pos) + self.emb_time(dts.unsqueeze(-1))
        pri = None
        if side_info is not None and self.ablation != "no_sideinfo":
            base = self.side_proc(side_info).mean(dim=1)
            if self.disent is not None:
                shared, pri = self.disent(side_info)
            else:
                shared, pri = base, None
            st = torch.stack([base, shared], 1)
            f, _ = self.side_attn(st, st, st)
            x = x + f.mean(dim=1).unsqueeze(1)
        out  = self.transformer(inputs_embeds=x).last_hidden_state
        main = out[:,0,:]
        proj = nn.functional.normalize(self.proj_head(main), dim=-1)
        logits = self.pred_head(main)
        ml = None
        if self.training and self.mask_rate > 0:
            m = (torch.rand(B, L, device=seqs.device) < self.mask_rate)
            m[:,0] = False
            if m.any():
                part = out[m]
                ml   = nn.functional.cross_entropy(self.mask_head(part), seqs[m])
        return logits, proj, self.tau, pri, ml

class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.3, label_smoothing=0.1):
        super().__init__()
        self.alpha        = alpha
        self.ls           = label_smoothing
        self.ortho_weight = 0.1
    def ce(self, logits, tgt):
        n  = logits.size(-1)
        oh = torch.zeros_like(logits).scatter(1, tgt.unsqueeze(1), 1)
        oh = oh*(1-self.ls) + self.ls/n
        lp = nn.functional.log_softmax(logits, dim=-1)
        return -(oh*lp).sum(-1).mean()
    def contrast(self, pos, negs, tau):
        tau = tau.clamp(min=0.01)
        sim = torch.cosine_similarity(pos.unsqueeze(1), negs, dim=-1)
        return -torch.log(torch.exp(sim/tau).mean(-1)+1e-8).mean()
    def forward(self, logits, tgt, pos, negs, tau, ortho=None):
        l1 = self.ce(logits, tgt)
        l2 = self.contrast(pos, negs, tau)
        L  = (1-self.alpha)*l1 + self.alpha*l2
        if ortho is not None:
            L += self.ortho_weight * ortho
        return L

visual_cfg = BertConfig(
    hidden_size=768, num_hidden_layers=8, num_attention_heads=8,
    intermediate_size=768*4, hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1, output_attentions=True
)

class EnhancedFusionRecommenderWithVisuals(EnhancedFusionRecommenderWithDisentanglement):
    def __init__(self, *a, **k):
        super().__init__(*a, **k)
        self.transformer = BertModel(visual_cfg)
    def forward(self, seqs, dts, side_info=None):
        logits, proj, tau, pri, ml = super().forward(seqs, dts, side_info)
        B, L = seqs.shape
        pos    = torch.arange(L, device=seqs.device).unsqueeze(0)
        x      = self.emb_item(seqs) + self.emb_pos(pos) + self.emb_time(dts.unsqueeze(-1))
        if side_info is not None:
            base, _ = self.disent(side_info)
            shared, _ = base, None
            st        = torch.stack([base, shared],1)
            f, _      = self.side_attn(st, st, st)
            x         = x + f.mean(dim=1).unsqueeze(1)
        out = self.transformer(inputs_embeds=x)
        return logits, proj, tau, pri, ml, out.attentions

# === Training, Evaluation & Ablation ===
def enhanced_evaluate(model, loader, device, use_side_info=False, side_info_matrices=None, zero_time=False):
    model.eval()
    hr_list, ndcg_list, p_list, mrr_list = [], [], [], []
    with torch.no_grad():
        for batch in loader:
            seqs, dts, tgt = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            if zero_time:
                dts = torch.zeros_like(dts)
            if use_side_info:
                si = {n: get_batch_side_info(seqs, mtx) for n, mtx in side_info_matrices.items()}
                logits, *_ = model(seqs, dts, side_info=si)
            else:
                logits, *_ = model(seqs, dts)
            topk = torch.topk(logits, 5, dim=1)[1]
            bs    = seqs.size(0)
            hr_b=ndcg_b=p_b=mrr_b=0.0
            for i in range(bs):
                t0 = tgt[i].item()
                preds = topk[i].tolist()
                if t0 in preds:
                    r = preds.index(t0)+1
                    hr_b   += 1
                    ndcg_b += 1/math.log2(r+1)
                    p_b    += 1.0/5
                    mrr_b  += 1.0/r
            hr_list.append(hr_b/bs)
            ndcg_list.append(ndcg_b/bs)
            p_list.append(p_b/bs)
            mrr_list.append(mrr_b/bs)
    def ms(x): return (np.mean(x), np.std(x, ddof=1) if len(x)>1 else 0.0)
    hr_m,hr_s      = ms(hr_list)
    ndcg_m,ndcg_s  = ms(ndcg_list)
    p_m,p_s        = ms(p_list)
    mrr_m,mrr_s    = ms(mrr_list)
    return {
        'hr':hr_m, 'hr_std':hr_s,
        'ndcg':ndcg_m, 'ndcg_std':ndcg_s,
        'precision':p_m, 'precision_std':p_s,
        'mrr':mrr_m, 'mrr_std':mrr_s
    }

def run_training(hp, train, val, si_mats, dev, num_items, si_dims, use_si=True, mask_rate_override=None, lambda_aux_override=None, ortho_weight=0.1, zero_time=False):
    mask_rate   = mask_rate_override if mask_rate_override is not None else hp['mask_rate']
    lambda_aux  = lambda_aux_override if lambda_aux_override is not None else hp['lambda_aux']
    model = EnhancedFusionRecommenderWithDisentanglement(
        num_items=num_items, side_info_dims=si_dims,
        lambda_aux=lambda_aux, mask_rate=mask_rate
    ).to(dev)
    opt    = AdamW(model.parameters(), lr=hp['lr'], weight_decay=hp['weight_decay'])
    total  = len(train) * hp['num_epochs']
    sch    = get_cosine_schedule_with_warmup(opt, int(0.1*total), total)
    crit   = CombinedLoss(alpha=0.3, label_smoothing=0.1)
    scaler = GradScaler()
    best, no_imp = 0, 0

    for _ in range(hp['num_epochs']):
        torch.cuda.empty_cache()
        model.train()
        for batch in train:
            seqs, dts, tgt = batch[0].to(dev), batch[1].to(dev), batch[2].to(dev)
            if zero_time:
                dts = torch.zeros_like(dts)
            negs = batch[3].to(dev) if len(batch)==4 else None
            si   = {n:get_batch_side_info(seqs, m) for n,m in si_mats.items()} if use_si else None

            opt.zero_grad()
            with autocast():
                logits, pos, tau, priv, ml = model(seqs, dts, side_info=si)
                if negs is not None:
                    with torch.no_grad():
                        neg_emb = nn.functional.normalize(model.emb_item(negs), dim=-1)
                    loss = (1-crit.alpha)*crit.ce(logits, tgt) + crit.alpha*crit.contrast(pos, neg_emb, tau)
                else:
                    loss = crit.ce(logits, tgt)
                if priv and ortho_weight>0:
                    loss += ortho_weight * orthogonality_loss(priv)
                if ml is not None:
                    loss += lambda_aux * ml

            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            scaler.step(opt)
            scaler.update()
            sch.step()

        val_m = enhanced_evaluate(model, val, dev, use_side_info=use_si, side_info_matrices=si_mats, zero_time=zero_time)
        if val_m['ndcg'] > best:
            best, no_imp = val_m['ndcg'], 0
        else:
            no_imp += 1
        if no_imp >= hp['patience']:
            break

    return best, model.state_dict()

def run_training_ablation(hp, train, val, si_mats, dev, num_items, si_dims, ablation):
    model = EnhancedFusionRecommenderWithDisentanglement(
        num_items=num_items,
        side_info_dims=si_dims,
        lambda_aux=hp['lambda_aux'],
        mask_rate=hp['mask_rate'],
        ablation=ablation
    ).to(dev)
    opt    = AdamW(model.parameters(), lr=hp['lr'], weight_decay=hp['weight_decay'])
    total  = len(train) * hp['num_epochs']
    sch    = get_cosine_schedule_with_warmup(opt, int(0.1*total), total)
    crit   = CombinedLoss(alpha=0.3, label_smoothing=0.1)
    scaler = GradScaler()
    best, no_imp = 0, 0

    for _ in range(hp['num_epochs']):
        model.train()
        for batch in train:
            seqs, dts, tgt = batch[0].to(dev), batch[1].to(dev), batch[2].to(dev)
            negs = batch[3].to(dev) if len(batch)==4 else None
            si   = {n:get_batch_side_info(seqs, m) for n,m in si_mats.items()} if (ablation!="no_sideinfo") else None

            opt.zero_grad()
            with autocast():
                logits, pos, tau, priv, ml = model(seqs, dts, side_info=si)
                loss = (1-crit.alpha)*crit.ce(logits, tgt) + crit.alpha*crit.contrast(pos, (nn.functional.normalize(model.emb_item(negs), dim=-1) if negs is not None else pos), tau)
                if priv is not None:
                    loss += model.ortho_weight * orthogonality_loss(priv)
                if ml is not None:
                    loss += model.lambda_aux * ml

            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            scaler.step(opt)
            scaler.update()
            sch.step()

        val_m = enhanced_evaluate(
            model, val, dev,
            use_side_info=(ablation!="no_sideinfo"),
            side_info_matrices=si_mats,
            zero_time=(ablation=="no_time")
        )
        if val_m['ndcg'] > best:
            best, no_imp = val_m['ndcg'], 0
        else:
            no_imp += 1
        if no_imp >= hp['patience']:
            break

    return best, model.state_dict()

# === Part 5/5: main_all with 10 seeds & bootstrap CIs ===
def main_all():
    start_time = time.time()
    csv = "MOOCCube/MOOCQA/tuples/rel_user_course.csv"

    # Data pipelines
    tr_ds = EnhancedMOOCDataset(csv, split="train", neg_samples=5)
    vl_ds = MOOCubeCourseDataset(csv, split="val")
    te_ds = MOOCubeCourseDataset(csv, split="test")
    tr_ld = DataLoader(tr_ds, batch_size=32, shuffle=True,  collate_fn=collate_fn, num_workers=4, pin_memory=True)
    vl_ld = DataLoader(vl_ds, batch_size=32, shuffle=False, collate_fn=collate_fn, num_workers=4, pin_memory=True)
    te_ld = DataLoader(te_ds, batch_size=32, shuffle=False, collate_fn=collate_fn, num_workers=4, pin_memory=True)
    dev   = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Side-info
    course_j  = load_json_file("MOOCCube/entities/course.json")
    teacher_j = load_json_file("MOOCCube/entities/teacher.json")
    act_j     = merge_activity_features([
        "MOOCCube/additional_information/prerequisite_prediction.json",
        "MOOCCube/additional_information/concept_infomation.json",
        "MOOCCube/additional_information/user_video_act.json"
    ])
    c_m_cpu = build_feature_matrix(course_j, tr_ds.i2i)
    t_m_cpu = build_feature_matrix(teacher_j, tr_ds.i2i)
    a_m_cpu = build_feature_matrix(act_j, tr_ds.i2i)
    si_mats_cpu = {'course':c_m_cpu, 'teacher':t_m_cpu, 'activity':a_m_cpu}
    si_mats     = {k: m.to(dev) for k,m in si_mats_cpu.items()}
    num_items = len(tr_ds.i2i)
    si_dims   = {'course':32,'teacher':32,'activity':32}

    # Hyperparam search
    hparam_grid = [
        {'lr':1e-5,'weight_decay':0.01,'num_epochs':80,'patience':10,'lambda_aux':0.1,'mask_rate':0.1},
        {'lr':1e-5,'weight_decay':0.001,'num_epochs':80,'patience':10,'lambda_aux':0.1,'mask_rate':0.1},
        {'lr':3e-5,'weight_decay':0.01,'num_epochs':80,'patience':10,'lambda_aux':0.1,'mask_rate':0.1},
        {'lr':3e-5,'weight_decay':0.001,'num_epochs':80,'patience':10,'lambda_aux':0.1,'mask_rate':0.1},
    ]
    best_ndcg, best_hp, best_state = 0, None, None
    for hp in hparam_grid:
        ndcg, state = run_training(hp, tr_ld, vl_ld, si_mats, dev, num_items, si_dims, use_si=True)
        if ndcg > best_ndcg:
            best_ndcg, best_hp, best_state = ndcg, hp, state

    # Final 10-seed evaluation
    seeds = [42, 123, 999, 2025, 31415, 271828, 314159, 161803, 27182, 141421]
    all_results = []
    for seed in seeds:
        set_seed(seed)
        _, state = run_training(best_hp, tr_ld, vl_ld, si_mats, dev, num_items, si_dims, use_si=True)
        model = EnhancedFusionRecommenderWithDisentanglement(
            num_items=num_items, side_info_dims=si_dims,
            lambda_aux=best_hp['lambda_aux'], mask_rate=best_hp['mask_rate']
        ).to(dev)
        model.load_state_dict(state)
        m = enhanced_evaluate(model, te_ld, dev, use_side_info=True, side_info_matrices=si_mats)
        all_results.append(m)

    # Bootstrap 95% CIs
    hr_lower, hr_upper, hr_mean       = bootstrap_ci([r['hr']   for r in all_results])
    ndcg_lower, ndcg_upper, ndcg_mean = bootstrap_ci([r['ndcg'] for r in all_results])
    p_lower, p_upper, p_mean          = bootstrap_ci([r['precision'] for r in all_results])
    mrr_lower, mrr_upper, mrr_mean    = bootstrap_ci([r['mrr'] for r in all_results])

    print(f"Final (over seeds {seeds}) → "
          f"HR@5:   {hr_mean:.4f} [{hr_lower:.4f}, {hr_upper:.4f}], "
          f"NDCG@5: {ndcg_mean:.4f} [{ndcg_lower:.4f}, {ndcg_upper:.4f}], "
          f"P@5:    {p_mean:.4f} [{p_lower:.4f}, {p_upper:.4f}], "
          f"MRR@5:  {mrr_mean:.4f} [{mrr_lower:.4f}, {mrr_upper:.4f}]")

    # Ablation studies
    print("=== Ablation studies ===")
    for ab in ["no_sideinfo","no_masked","no_disentanglement","no_time","no_orthogonality"]:
        _, st = run_training_ablation(best_hp, tr_ld, vl_ld, si_mats, dev, num_items, si_dims, ab)
        model = EnhancedFusionRecommenderWithDisentanglement(
            num_items=num_items, side_info_dims=si_dims,
            lambda_aux=best_hp['lambda_aux'], mask_rate=best_hp['mask_rate']
        ).to(dev)
        model.load_state_dict(st, strict=False)
        m = enhanced_evaluate(model, te_ld, dev,
                              use_side_info=(ab!="no_sideinfo"),
                              side_info_matrices=si_mats,
                              zero_time=(ab=="no_time"))
        print(f"Ablation {ab:<15} → "
              f"HR@5: {m['hr']:.4f}±{m['hr_std']:.4f}, "
              f"NDCG@5: {m['ndcg']:.4f}±{m['ndcg_std']:.4f}, "
              f"P@5: {m['precision']:.4f}±{m['precision_std']:.4f}, "
              f"MRR@5: {m['mrr']:.4f}±{m['mrr_std']:.4f}")

    # Visualizations & SHAP/IG
    entropy_records = []
    torch.cuda.empty_cache()
    vis_model = EnhancedFusionRecommenderWithVisuals(
        num_items=num_items,
        side_info_dims=si_dims,
        lambda_aux=best_hp['lambda_aux'],
        mask_rate=best_hp['mask_rate']
    ).cpu()
    vis_model.load_state_dict(best_state, strict=False)
    vl_vis = DataLoader(vl_ds, batch_size=8, shuffle=False, collate_fn=collate_fn)
    seqs, dts, tgt     = next(iter(vl_vis))
    side_cpu           = {k: get_batch_side_info(seqs, m) for k,m in si_mats_cpu.items()}
    logits, proj, tau, priv, ml, attn_weights = vis_model(seqs, dts, side_info=side_cpu)

    num_layers = len(attn_weights)
    for layer in (0, num_layers//2, num_layers-1):
        ents, hmax_idx, hmin_idx = compute_layer_head_entropies(attn_weights, layer)
        for head_i, ent in enumerate(ents):
            entropy_records.append({'layer': layer+1, 'head': head_i+1, 'entropy': ent})
        plot_layer_entropy(attn_weights, layer)
        print(f"Layer {layer+1}: max ent head={hmax_idx+1}, min ent head={hmin_idx+1}")
        plot_attention_heatmap(attn_weights, layer, hmax_idx, title="(max ent)")
        plot_attention_heatmap(attn_weights, layer, hmin_idx, title="(min ent)")

    pd.DataFrame(entropy_records).to_excel("entropy_values.xlsx", index=False)
    print("Entropy values saved to entropy_values.xlsx")

    # SHAP/IG example case
    try:
        print("\n=== SHAP/IG örnek vaka incelemesi ===")
        seqs, dts, tgt   = next(iter(vl_vis))
        seq              = seqs[0:1].to(dev)
        dts_seq          = dts[0:1].to(dev)
        mask_pos         = seq.size(1) - 1
        side_one         = {k: v[0:1].to(dev) for k,v in side_cpu.items()}

        ig_model = EnhancedFusionRecommenderWithVisuals(
            num_items=num_items,
            side_info_dims=si_dims,
            lambda_aux=best_hp['lambda_aux'],
            mask_rate=best_hp['mask_rate']
        ).to(dev)
        ig_model.load_state_dict(best_state)
        ig_model.eval()
        for p in ig_model.parameters(): p.requires_grad_(False)

        item_emb  = ig_model.emb_item(seq)
        pos_idx   = torch.arange(seq.size(1), device=dev).unsqueeze(0)
        pos_emb   = ig_model.emb_pos(pos_idx)
        time_emb  = ig_model.emb_time(dts_seq.unsqueeze(-1))
        embed_in  = (item_emb + pos_emb + time_emb).detach().requires_grad_(True)

        def emb_forward(emb, side=side_one):
            x         = emb
            base, _   = ig_model.side_proc(side).mean(1), None
            shared, _ = ig_model.disent(side)
            fuse, _   = ig_model.side_attn(
                torch.stack([base, shared],1),
                torch.stack([base, shared],1),
                torch.stack([base, shared],1)
            )
            x         = x + fuse.mean(1).unsqueeze(1)
            out       = ig_model.transformer(inputs_embeds=x).last_hidden_state[:,0,:]
            logits    = ig_model.pred_head(out)
            return logits[:, mask_pos]

        ig = IntegratedGradients(emb_forward)
        baseline = torch.zeros_like(embed_in)
        atts, delta = ig.attribute(
            inputs=embed_in,
            baselines=baseline,
            return_convergence_delta=True,
            n_steps=50
        )
        ig_scores = atts[0].abs().sum(-1).detach().cpu().numpy()
        contrib   = ig_scores[mask_pos]
        print(f"Mask position contribution to MRR (IG): +{contrib:.4f}")

        plt.figure()
        plt.bar(range(len(ig_scores)), ig_scores)
        plt.xlabel("Sequence Position")
        plt.ylabel("Total IG Contribution")
        plt.title("Position-wise IG Contributions (Embedding-based)")
        plt.show()

    except Exception as e:
        print("SHAP/IG example error:", e)

    end_time = time.time()
    print(f"🏁 Total runtime: {(end_time - start_time)/60:.2f} min")

if __name__ == "__main__":
    main_all()
